In [1]:
import json
import pandas as pd
import re
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import os
from datetime import datetime

In [3]:
# Retrieve the output data from the JSON file
with open("mobile_de/output.json", mode="r", encoding='utf-8') as f:
    df_data_all_car_brands = json.load(f)
    df_data_all_car_brands = pd.DataFrame(df_data_all_car_brands)
    f.close()

# Drop the duplicates because the mileage filters applied above could have produced duplicates
df_data_all_car_brands = df_data_all_car_brands.drop_duplicates(["url_to_crawl"])

# Step 1: Clean the data
df_data_all_car_brands_cleaned = df_data_all_car_brands.copy()
df_data_all_car_brands_cleaned.replace(to_replace="", value=None, inplace=True)
df_data_all_car_brands_cleaned["leistung"] = df_data_all_car_brands_cleaned["leistung"].apply(lambda x: int(re.findall(pattern="(?<=\().*(?=\sPS)", string=x)[0].replace(".", "")) if x is not None else x)
df_data_all_car_brands_cleaned["preis"] = df_data_all_car_brands_cleaned["preis"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
df_data_all_car_brands_cleaned["kilometer"] = df_data_all_car_brands_cleaned["kilometer"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
df_data_all_car_brands_cleaned["fahrzeughalter"] = df_data_all_car_brands_cleaned["fahrzeughalter"].apply(lambda x: int(x) if x is not None else x)
df_data_all_car_brands_cleaned["standort"] = df_data_all_car_brands_cleaned["standort"].apply(lambda x: re.findall(pattern="[A-za-z]+(?=-)", string=x)[0] if x is not None else x)
df_data_all_car_brands_cleaned["crawled_timestamp"] = datetime.now()

In [5]:
# Step 2: Upload to bigquery
# First, set the credentials
key_path_home_dir = os.path.expanduser("~") + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path_home_dir, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
bq_client = bigquery.Client(project="web-scraping-371310", credentials=credentials)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("marke", "STRING"),
        bigquery.SchemaField("modell", "STRING"),
        bigquery.SchemaField("variante", "STRING"),
        bigquery.SchemaField("titel", "STRING"),
        bigquery.SchemaField("form", "STRING"),
        bigquery.SchemaField("fahrzeugzustand", "STRING"),
        bigquery.SchemaField("leistung", "FLOAT64"),
        bigquery.SchemaField("getriebe", "STRING"),
        bigquery.SchemaField("farbe", "STRING"),
        bigquery.SchemaField("preis", "INT64"),
        bigquery.SchemaField("kilometer", "FLOAT64"),
        bigquery.SchemaField("erstzulassung", "STRING"),
        bigquery.SchemaField("fahrzeughalter", "FLOAT64"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("fahrzeugbeschreibung", "STRING"),
        bigquery.SchemaField("url_to_crawl", "STRING"),
        bigquery.SchemaField("page_rank", "INT64"),
        bigquery.SchemaField("total_num_pages", "INT64"),
        bigquery.SchemaField("crawled_timestamp", "TIMESTAMP"),
    ]
)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

# Upload the table
bq_client.load_table_from_dataframe(
    dataframe=df_data_all_car_brands_cleaned,
    destination="web-scraping-371310.crawled_datasets.lukas_mobile_de",
    job_config=job_config
).result()

## Data cleaning Porsche 992 GT3

In [8]:
query = """
    SELECT *
    FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`
    WHERE crawled_timestamp = (SELECT MAX(crawled_timestamp) FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`)
"""
df = pd.DataFrame(bq_client.query(query).to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type="tqdm_notebook"))

Query is running:   0%|          |

Downloading:   0%|          |

In [52]:
print(f"The length of the dataset is: {len(df)}")

The length of the dataset is: 52479


#### Filter for Porsche 992

In [53]:
df_clean_1 = df[
    (df["marke"]=="Porsche") &
    (df["modell"]=="992")
].reset_index(drop=True)

In [54]:
df_clean_1

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992,Cabrio / Roadster,None,385.0,None,Schwarz Metallic,125000,33000.0,05/2020,2.0,DE,"Sehr gepflegter Garagen Wagen, keine Tiere, Ni...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992,Cabrio / Roadster,None,385.0,None,Schwarz Metallic,125000,33000.0,05/2020,2.0,DE,"Sehr gepflegter Garagen Wagen, keine Tiere, Ni...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 Carrera,Sportwagen / Coupé,None,385.0,Automatik,Grau,129900,25938.0,01/2020,1.0,NL,Informationen auf Deutsch: \n \n Original nied...,https://suchen.mobile.de/fahrzeuge/details.htm...,11,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 Targa 4,Cabrio / Roadster,None,385.0,Automatik,Blau Metallic,149500,27400.0,11/2020,NaN,BE,Porsche 992 Targa 4 2020\nFirst registration 2...,https://suchen.mobile.de/fahrzeuge/details.htm...,14,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 Turbo S,Sportwagen / Coupé,None,650.0,Automatik,Silber Metallic,207894,55000.0,09/2020,2.0,NL,Informationen auf Deutsch: \n \n \n \nWeitere ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2247,Porsche,992,None,Porsche 992 991/992 Carrera Cabriolet S*Sporta...,Cabrio / Roadster,None,450.0,Automatik,Grau Metallic,142980,694.0,10/2022,1.0,DE,"Wir freuen uns sehr, dass wir mit unserem Ange...",https://suchen.mobile.de/fahrzeuge/details.htm...,4,50,2024-03-16 21:44:09.106746+00:00
2248,Porsche,992,None,Porsche 992 Perfekter Traum:992 Carrera GTS Co...,Sportwagen / Coupé,Unfallfrei,480.0,Automatik,Rot Metallic,189000,900.0,10/2023,1.0,DE,"Neuwertiger 992 GTS, 1. Hand, perfekt ohne Geb...",https://suchen.mobile.de/fahrzeuge/details.htm...,21,50,2024-03-16 21:44:09.106746+00:00
2249,Porsche,992,None,"Porsche 992 Traumschöner 992 Targa 4S, Vollaus...",Cabrio / Roadster,Unfallfrei,450.0,Automatik,Orange Metallic,179000,2456.0,03/2022,1.0,DE,"Sammlerzustand, 992 Targa 4S, ohne Gebrauchssp...",https://suchen.mobile.de/fahrzeuge/details.htm...,5,50,2024-03-16 21:44:09.106746+00:00
2250,Porsche,992,None,Porsche 992 911 -992 Carrera 4 GTS Cabrio - N...,Cabrio / Roadster,Unfallfrei,480.0,None,Schwarz,178700,4800.0,10/2023,1.0,DE,4.800 km !!-WIE NEU!!\n1.HAND DEUTSCHES FAHRZE...,https://suchen.mobile.de/fahrzeuge/details.htm...,15,50,2024-03-16 21:44:09.106746+00:00


#### Add fahrzeugbeschreibung_mod column to replace None values in that column with an empty string

In [96]:
# Add fahrzeugbeschreibung_mod column to replace None values in that column with an empty string
df_clean_1["fahrzeugbeschreibung_mod"] = df_clean_1["fahrzeugbeschreibung"].apply(lambda x: "" if x is None else x)

#### Filter for GT3 or GT 3

In [55]:
df_clean_2 = df_clean_1.loc[
    (df_clean_1["titel"].str.lower().str.contains("gt3|gt 3")) | (df["fahrzeugbeschreibung_mod"].str.lower().str.contains("gt3|gt 3"))
].reset_index(drop=True)
df_clean_2

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 CLUBSPORT PTS ACID GREEN - VAT,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,220680,35600.0,09/2021,NaN,FR,PORSCHE 992 GT3 4.0 510 CLUBSPORT PTS ACID GRE...,https://suchen.mobile.de/fahrzeuge/details.htm...,13,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Sportwagen / Coupé,None,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,"Sportwagen / Coupé, Neufahrzeug",Unfallfrei,510.0,Automatik,Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
340,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
341,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
342,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Filter for DE

In [56]:
df_clean_3 = df_clean_2[
    (df_clean_2["standort"].str.lower().str.contains("de"))
].reset_index(drop=True)
df_clean_3

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Sportwagen / Coupé,None,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,"Sportwagen / Coupé, Neufahrzeug",Unfallfrei,510.0,Automatik,Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Amend the `form` column

In [57]:
# Spalte E = form = Wenn Sportwagen/Coupe, oder Sportwagen/Coupe, Jahreswagen, oder Sportwagen/Coupe, Neufahrzeug, oder Sportwagen/Coupe, Tageszulassung, oder
# Sportwagen/Coupe, Vorführfahrzeug abgebildet wird, dann auf „Coupé“ ändern!

df_clean_4 = df_clean_3.copy()

# Write a function that checks each value in the column and returns "True" if the value contains "sportwagen/coupé" and "False" otherwise.
def amend_form_col_porsche_992_gt3(x):
    if x.find("sportwagen/coup") != -1:
        return "Coupé"
    else:
        return x
    
# Remove the spaces from the form column and convert to lower case
df_clean_4["form_mod"] = df_clean_4["form"].apply(lambda x: x.replace(" ", "").lower())

# Apply the function amend_form_col on form using the apply method
df_clean_4["form"] = df_clean_4["form_mod"].apply(amend_form_col_porsche_992_gt3)

# Drop the form_mod column
df_clean_4 = df_clean_4.drop("form_mod", axis=1)

# Display the data frame
df_clean_4

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,None,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Coupé,Unfallfrei,510.0,Automatik,Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,Coupé,Unfallfrei,510.0,Automatik,Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Amend the `fahrzeugzustand` column

In [58]:
# Spalte F = fahrzeugzustand = Wenn Unfallfrei, Nicht fahrtauglich, oder (Leere) abgebildet wird, dann auf “Unfallfrei“ ändern
df_clean_5 = df_clean_4.copy()
def amend_fahrzeugzustand_col_porsche_992_gt3(x):
    if x == "" or x is None:
        return "Unfallfrei"
    elif x.lower().find("unfallfrei") != -1:
        return "Unfallfrei"
    else:
        return x
df_clean_5["fahrzeugzustand"] = df_clean_5["fahrzeugzustand"].apply(amend_fahrzeugzustand_col_porsche_992_gt3)
df_clean_5

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Coupé,Unfallfrei,510.0,Automatik,Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,Coupé,Unfallfrei,510.0,Automatik,Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Amend the `Kilometer` column

In [59]:
# Spalte K = kilometer = Wenn (Leere) abgebildet wird, dann auf „1“ ändern. (Info: Sind oftmals Neuwagen und haben Werkskilometer)
df_clean_6 = df_clean_5.copy()
def amend_kilometer_col_porsche_992_gt3(x):
    if x == "" or x is None:
        return 1
    else:
        return x
df_clean_6["kilometer"] = df_clean_6["kilometer"].apply(amend_kilometer_col_porsche_992_gt3)
df_clean_6

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Coupé,Unfallfrei,510.0,Automatik,Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,Coupé,Unfallfrei,510.0,Automatik,Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Amend the `Getriebe` col

In [60]:
# Spalte H = getriebe = Wenn getriebe Automatik, dann ändere auf „7-Gang Doppelkupplungsgetriebe (PDK)“
# Spalte H = getriebe = Wenn getriebe Schaltgetriebe, dann ändere auf „6-Gang-GT-Sportschaltgetriebe“.
df_clean_7 = df_clean_6.copy()
def amend_getriebe_col_stg_1_porsche_992_gt3(x):
    if x == "" or x is None:
        return x
    elif x.lower().find("automatik") != -1:
        return "7-Gang Doppelkupplungsgetriebe (PDK)"
    elif x.lower().find("schaltgetriebe") != -1:
        return "6-Gang-GT-Sportschaltgetriebe"
    else:
        return x
    
df_clean_7["getriebe"] = df_clean_7["getriebe"].apply(amend_getriebe_col_stg_1_porsche_992_gt3)
df_clean_7

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Amend the `Marke` col based on the `titel` col

In [61]:
# Spalte D -> A = titel = Wenn im titel „Techart“ steht, dann ändere marke (Spalte A) auf “Porsche/Techart“
# Spalte D -> A = titel = Wenn im titel „Manthey“ steht, dann ändere marke (Spalte A) auf “Porsche/Manthey Performance“
# Spalte D -> A = titel = Wenn im titel „Cup“ steht, dann ändere marke (Spalte A) auf “Porsche/Racing“
df_clean_8 = df_clean_7.copy()
def amend_marke_col_porsche_992_gt3(x, y, replacement_word):
    if x.lower().find(y.lower()) != -1:
        return "Porsche | " + replacement_word

df_clean_8["marke"] = df_clean_8.apply(lambda x: amend_marke_col_porsche_992_gt3(x["titel"], "Techart", "Techart") if x["titel"].lower().find("techart") != -1 else x["marke"], axis=1)
df_clean_8["marke"] = df_clean_8.apply(lambda x: amend_marke_col_porsche_992_gt3(x["titel"], "Manthey", "Manthey") if x["titel"].lower().find("manthey") != -1 else x["marke"], axis=1)
df_clean_8["marke"] = df_clean_8.apply(lambda x: amend_marke_col_porsche_992_gt3(x["titel"], "Cup", "Racing") if x["titel"].lower().find("cup") != -1 else x["marke"], axis=1)
df_clean_8

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


#### Amend the `variante` column

In [62]:
# Rule 1: Spate A|B|D|G|O verändern Spalte C = variante = Wenn marke Porsche, und modell 992, und leistung 510, und entweder im titel Touring oder in fahrzeugbeschreibung Touring
# steht, dann ändere in Spalte C auf “GT3 mit Touring-Paket“

# Rule 2: Spate A|B|D|G|O verändern Spalte C = variante = Wenn marke Porsche, und Modell 992, und leistung 510, und Spalte C nicht GT3 mit Touring-Paket steht, dann ändere auf “GT3“.

df_clean_9 = df_clean_8.copy()
def amend_variante_col_porsche_992_gt3(x):
    if x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510 and (x["titel"].lower().find("touring") != -1 or x["fahrzeugbeschreibung_mod"].lower().find("touring") != -1):
        return "GT3 mit Touring-Paket"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510 and (x["titel"].lower().find("touring") == -1 and x["fahrzeugbeschreibung_mod"].lower().find("touring") == -1):
        return "GT3"
    else:
        return x["variante"]

# Rule 1: Spate H = getriebe = Wenn marke Porsche, und Modell 992, und leistung 510, und Variante GT3 oder GT3 mit Touring-Paket, und entweder im titel PDK oder im fahrzeugbeschreibung PDK steht, und getriebe auf (Leere), dann ändere auf “7-Gang Doppelkupplungsgetriebe (PDK)“

# Rule 2: Spate H = getriebe = Wenn marke Porsche, und Modell 992, und leistung 510, und Variante GT3 oder GT3 mit Touring-Paket, und entweder im titel 6-Gang oder im fahrzeugbeschreibung 6-Gang steht, und getriebe auf (Leere), dann ändere auf "6-Gang-GT-Sportschaltgetriebe"
def amend_getriebe_col_stg_2_porsche_992_gt3(x):
    if x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510\
    and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and (x["titel"].lower().find("pdk") != -1 or x["fahrzeugbeschreibung_mod"].lower().find("pdk") != -1) and (x["getriebe"] == "" or x["getriebe"] is None):
        return "7-Gang Doppelkupplungsgetriebe (PDK)"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510\
    and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and (x["titel"].lower().find("6-gang") != -1 or x["fahrzeugbeschreibung_mod"].lower().find("6-gang") != -1) and (x["getriebe"] == "" or x["getriebe"] is None):
        return "6-Gang-GT-Sportschaltgetriebe"
    else:
        return x["getriebe"]

# Rule 3: Spate H = getriebe = Wenn marke Porsche, und Modell 992, und leistung 510, und Variante GT3 oder GT3 mit Touring-Paket, und getriebe auf (Leere), dann ändere auf “7-Gang Doppelkupplungsgetriebe (PDK)“.“.
def amend_getriebe_col_stg_3_porsche_992_gt3(x):
    if x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510\
    and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and (x["getriebe"] == "" or x["getriebe"] is None):
        return "7-Gang Doppelkupplungsgetriebe (PDK)"
    else:
        return x["getriebe"]
    
# Apply the functions
df_clean_9["variante"] = df_clean_9.apply(
    lambda x: amend_variante_col_porsche_992_gt3(x), axis=1
)

# Getriebe Stg 2
df_clean_9["getriebe"] = df_clean_9.apply(lambda x: amend_getriebe_col_stg_2_porsche_992_gt3(x), axis=1)

# Getriebe Stg 3
df_clean_9["getriebe"] = df_clean_9.apply(lambda x: amend_getriebe_col_stg_3_porsche_992_gt3(x), axis=1)

In [63]:
display(df_clean_9[df_clean_9["getriebe"].isnull()])

display(df_clean_9["getriebe"].value_counts())

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
142,Porsche,992,None,"Porsche 992 GT3 RS, Weissach Paket, Lift, PCCB",Coupé,Unfallfrei,525.0,None,Gelb,395000,37.0,06/2023,1.0,DE,Zum Verkauf steht ein Porsche 992 GT3 RS Neuwa...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,50,2024-03-16 21:44:09.106746+00:00
175,Porsche,992,None,Porsche 992 GT3 RS *WEISSACH*CERAMIC*CLUBSPORT...,Coupé,Unfallfrei,525.0,None,Grau,404481,146.0,06/2023,NaN,DE,Porsche 992 911 GT3 RS WEISSACH CARBON IMMEDIA...,https://suchen.mobile.de/fahrzeuge/details.htm...,18,50,2024-03-16 21:44:09.106746+00:00
209,Porsche,992,None,"Porsche 992 GT3 RS CLUBSPORTPAKET,LIFTING,SCHA...",Coupé,Unfallfrei,525.0,None,Grau,359992,625.0,03/2023,NaN,DE,Traumhafter Porsche 992 GT3 RS Clubsportpaket...,https://suchen.mobile.de/fahrzeuge/details.htm...,14,50,2024-03-16 21:44:09.106746+00:00


getriebe
7-Gang Doppelkupplungsgetriebe (PDK)    252
6-Gang-GT-Sportschaltgetriebe            26
Name: count, dtype: int64

#### New column `ausstattung`

In [64]:
# Neue Spalte D = Wenn marke Porsche, und Modell 992, und Variante GT3 oder GT3 mit Touring-Paket, und form Coupe, und fahrzeugzustand unfallfrei, und Leistung 510, und im titel PTS steht, dann ändere auf „PTS“.
# Neue Spalte D = Wenn marke Porsche, und Modell 992, und Variante GT3 oder GT3 mit Touring-Paket, und form Coupe, und fahrzeugzustand unfallfrei, und Leistung 510, und entweder im titel PCCB oder im Fahrzeugbeschreibung PCCB steht, dann ändere auf „PCCB“. (Info: Wenn das modell schon die Ausstattung PTS hat und auch PCCB, dann ändere auf „PTS | PCCB“)
# Neue Spalte D = Wenn marke Porsche, und Modell 992, und Variante GT3 oder GT3 mit Touring-Paket, und form Coupe, und fahrzeugzustand unfallfrei, und Leistung 510, und entweder im titel Manufa oder im Fahrzeugbeschreibung Manufa steht, dann ändere auf „Exklusive Manufaktur“.

df_clean_10 = df_clean_9.copy()
def add_ausstattung_col_porsche_992_gt3(x):
    if x["marke"] == "Porsche" and x["modell"] == "992" and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and x["form"] == "Coupé" and x["fahrzeugzustand"].lower() == "unfallfrei" and x["leistung"] == 510\
    and (
        ("pts" in x["titel"].lower() and "pccb" in x["titel"].lower()) or\
        ("pts" in x["fahrzeugbeschreibung_mod"].lower() and "pccb" in x["fahrzeugbeschreibung_mod"].lower())
    ):
        return "PTS + PCCB"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and x["form"] == "Coupé" and x["fahrzeugzustand"].lower() == "unfallfrei" and x["leistung"] == 510\
    and (x["titel"].lower().find("pts") != -1):
        return "PTS"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and x["form"] == "Coupé" and x["fahrzeugzustand"] == "Unfallfrei" and x["leistung"] == 510\
    and (x["titel"].lower().find("pccb") != -1 or x["fahrzeugbeschreibung_mod"].lower().find("pccb") != -1):
        return "PCCB"
    else:
        return None

# Apply the function
df_clean_10["ausstattung"] = df_clean_10.apply(lambda x: add_ausstattung_col_porsche_992_gt3(x), axis=1)

# Move the Austattung column to be between "variante" and "titel"
austattung_col = df_clean_10.pop("ausstattung")
df_clean_10.insert(3, "ausstattung", austattung_col)

df_clean_10

,marke,modell,variante,ausstattung,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,GT3,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,177191,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-16 21:44:09.106746+00:00
1,Porsche,992,GT3,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
2,Porsche,992,GT3 mit Touring-Paket,PTS + PCCB,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-16 21:44:09.106746+00:00
3,Porsche,992,GT3,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-16 21:44:09.106746+00:00
4,Porsche,992,GT3,None,Porsche 992 (911) GT3 | Liftsystem | LED-Matri...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,200900,22100.0,12/2021,1.0,DE,AUSSTATTUNGSPAKETE\n Ablagepaket\n Interieur-...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,25,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,Porsche,992,GT3,None,Porsche 992 911 992*GT3*CLUBSPORT*LIFT*CARBON*...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,267750,50.0,None,0.0,DE,AURUM International\nPORSCHE 911 992 GT3 !!\nL...,https://suchen.mobile.de/fahrzeuge/details.htm...,38,50,2024-03-16 21:44:09.106746+00:00
277,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-16 21:44:09.106746+00:00
278,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-16 21:44:09.106746+00:00
279,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau Metallic,199770,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-16 21:44:09.106746+00:00


In [ ]:
# Drop the fahrzeugbeschreibung_mod column
df_clean_10 = df_clean_10.drop("fahrzeugbeschreibung_mod", axis=1)

In [65]:
# Save the data frame to a CSV file
df_clean_10.to_csv("porsche_992_gt3.csv", index=False)

### Data Cleaning Lamborghini Urus

In [135]:
query = """
    SELECT *
    FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`
    WHERE crawled_timestamp = (SELECT MAX(crawled_timestamp) FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`)
"""
df = pd.DataFrame(bq_client.query(query).to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type="tqdm_notebook"))

Query is running:   0%|          |

Downloading:   0%|          |

In [136]:
print(f"The length of the dataset is: {len(df)}")

The length of the dataset is: 52479


#### Filter for Lamborghini Urus in DE

In [137]:
df_clean_1 = df[
    (df["marke"]=="Lamborghini") &
    (df["modell"]=="Urus") &
    (df["standort"].str.lower().str.contains("de"))
].reset_index(drop=True)

In [138]:
df_clean_1

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Lamborghini,Urus,None,Lamborghini Urus,SUV / Geländewagen / Pickup,Unfallfrei,650.0,Automatik,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00
1,Lamborghini,Urus,None,Lamborghini Urus,SUV / Geländewagen / Pickup,Unfallfrei,650.0,Automatik,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00
2,Lamborghini,Urus,None,Lamborghini Urus,SUV / Geländewagen / Pickup,Unfallfrei,650.0,Automatik,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00
3,Lamborghini,Urus,None,Lamborghini Urus,SUV / Geländewagen / Pickup,Unfallfrei,650.0,Automatik,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00
4,Lamborghini,Urus,None,Lamborghini Urus,SUV / Geländewagen / Pickup,Unfallfrei,650.0,Automatik,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,"SUV / Geländewagen / Pickup, Neufahrzeug",Unfallfrei,666.0,Automatik,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00
219,Lamborghini,Urus,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...","SUV / Geländewagen / Pickup, Neufahrzeug",Unfallfrei,666.0,Automatik,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00
220,Lamborghini,Urus,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,"SUV / Geländewagen / Pickup, Neufahrzeug",Unfallfrei,666.0,Automatik,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00
221,Lamborghini,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,SUV / Geländewagen / Pickup,Unfallfrei,650.0,Automatik,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00


#### Add fahrzeugbeschreibung_mod column to replace None values in that column with an empty string

In [139]:
# Add fahrzeugbeschreibung_mod column to replace None values in that column with an empty string
df_clean_1["fahrzeugbeschreibung_mod"] = df_clean_1["fahrzeugbeschreibung"].apply(lambda x: "" if x is None else x)

#### Amend the `form` column

In [140]:
# Spalte E = form = Wenn SUV/Geländewagen/Pickup, oder SUV/Geländewagen/Pickup, Jahreswagen, oder SUV/Geländewagen/Pickup, Neufahrzeug, oder SUV/Geländewagen/Pickup, Tageszulassung, oder SUV/Geländewagen/Pickup, Vorführfahrzeug, oder Andere, oder Andere, Neufahrzeug, oder Sportwagen/Coupe, oder Sportwagen/Coupe, Neufahrzeug, oder Sportwagen/Coupe, Vorführfahrzeug abgebildet wird, dann auf „Super Sport Utility Vehicle“ ändern!
df_clean_2 = df_clean_1.copy()

# Write a function that checks each value in the column and returns "True" if the value contains "sportwagen/coupé" and "False" otherwise.
def amend_form_col_lamborghini_urus(x):
    if x.lower().find("suv") != -1 or x.lower().find("geländewagen") != -1\
    or x.lower().find("pickup") != -1 or x.lower().find("andere") != -1\
    or x.lower().find("sportwagen") != -1 or x.lower().find("coup") != -1:
        return "Super Sport Utility Vehicle"
    else:
        return x

# Apply the function amend_form_col on form using the apply method
df_clean_2["form"] = df_clean_2["form"].apply(amend_form_col_lamborghini_urus)

# Display the data frame
df_clean_2

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


#### Amend the `fahrzeugzustand` column

In [141]:
# Spalte F = fahrzeugzustand = Wenn Unfallfrei, Nicht fahrtauglich, oder (Leere) abgebildet wird, dann auf “Unfallfrei“ ändern
df_clean_3 = df_clean_2.copy()

def amend_fahrzeugzustand_col_lamborghini_urus(x):
    if x == "" or x is None:
        return "Unfallfrei"
    elif x.lower().find("unfallfrei") != -1:
        return "Unfallfrei"
    else:
        return x
df_clean_3["fahrzeugzustand"] = df_clean_3["fahrzeugzustand"].apply(amend_fahrzeugzustand_col_lamborghini_urus)
df_clean_3

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


In [142]:
df_clean_3["fahrzeugzustand"].value_counts()

fahrzeugzustand
Unfallfrei                                        219
Beschädigt, Unfallfahrzeug, Nicht fahrtauglich      2
Reparierter Unfallschaden                           1
Beschädigt                                          1
Name: count, dtype: int64

#### Amend the `kilometer` column

In [143]:
# Spalte K = kilometer = Wenn (Leere) abgebildet wird, dann auf „1“ ändern. (Info: Sind oftmals Neuwagen und haben Werkskilometer)
df_clean_4 = df_clean_3.copy()
def amend_kilometer_col_lamborghini_urus(x):
    if x == "" or x is None:
        return 1
    else:
        return x
df_clean_4["kilometer"] = df_clean_4["kilometer"].apply(amend_kilometer_col_lamborghini_urus)
df_clean_4

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,Automatik,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,Automatik,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


#### Amend the `Getriebe` col

In [144]:
# Spalte K = kilometer = Wenn (Leere) abgebildet wird, dann auf „1“ ändern. (Info: Sind oftmals Neuwagen und haben Werkskilometer)
df_clean_5 = df_clean_4.copy()
def amend_getriebe_col_stg_1_lamborghini_urus(x):
    if x == "" or x is None:
        return "8-Gang-Automatik-Getriebe"
    elif x.lower().find("automatik") != -1 or x.lower().find("halbautomatik") != -1 or x.lower().find("schaltgetriebe") != -1:
        return "8-Gang-Automatik-Getriebe"
    else:
        return x
    
df_clean_5["getriebe"] = df_clean_5["getriebe"].apply(amend_getriebe_col_stg_1_lamborghini_urus)
df_clean_5

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


In [145]:
df_clean_5["getriebe"].value_counts()

getriebe
8-Gang-Automatik-Getriebe    223
Name: count, dtype: int64

#### Amend the `Marke` col based on the `titel` col

In [146]:
# Spalte D -> A = titel = Wenn entweder im titel „gepanzert“ oder fahrzeugbeschreibung „gepanzert“ steht, dann ändere marke (Spalte A) auf “Lamborghini/Trasco“. (Info: Tuning Autos werden nicht berücksichtigt)
# Spalte D -> A = titel = Wenn entweder im titel „Mansory“oder fahrzeugbeschreibung Mansory steht, dann ändere marke (Spalte A) auf “Lamborghini/Mansory“.
# Spalte D -> A = titel = Wenn entweder im titel „Novitec“oder fahrzeugbeschreibung Novitec steht, dann ändere marke (Spalte A) auf “Lamborghini/Novitec“.
# Spalte D -> A = titel = Wenn entweder im titel „Keyvany“oder fahrzeugbeschreibung Keyvany steht, dann ändere marke (Spalte A) auf “Lamborghini/Keycany
# Spalte D -> A = titel = Wenn entweder im titel „Abt“oder fahrzeugbeschreibung Abt steht, dann ändere marke (Spalte A) auf “Lamborghini/Abt“.
df_clean_6 = df_clean_5.copy()
def amend_marke_col_lamborghini_urus(x, y, replacement_word):
    if x["titel"].lower().find(y.lower()) != -1 or x["fahrzeugbeschreibung_mod"].lower().find(y.lower()) != -1:
        return "Lamborghini | " + replacement_word
    else:
        return x["marke"]

lamborghini_urus_marke_dict = {
    "gepanzert": "Trasco",
    "mansory": "Mansory",
    "novitec": "Novitec",
    "keyvany": "Keyvany",
    "abt": "Abt"
}

for key, value in lamborghini_urus_marke_dict.items():
    df_clean_6["marke"] = df_clean_6.apply(lambda x: amend_marke_col_lamborghini_urus(x, key, value), axis=1)

df_clean_6

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


#### Amend the `variante` column

In [147]:
# Spalte A|B|D|G|O verändern Spalte C= variante = Wenn marke Lamborghini , und Modell Urus, und im titel Performante steht, dann ändere Variante auf Urus Performante.
# Spalte G = Leistung = Wenn variante Urus Performante,, dann auf 666 ändern

# Spate A|B|D|G|O verändern Spalte C= variante = Wenn marke Lamborghini , und Modell Urus, und variante (leere), und leistung 666, dann ändere Variante auf Urus S.

# Spate A|B|D|G|O verändern Spalte C= variante = Wenn marke Lamborghini , und Modell Urus, und variante (leere), und entweder leistung 650 oder leistung 649 oder Leistung 662, dann ändere Variante auf Urus.
# Spalte G = Leistung = Wenn variante Urus Performante,, dann auf 650 ändern

df_clean_7 = df_clean_6.copy()
def amend_variante_col_lamborghini_urus_stg_1(x):
    if x["marke"] == "Lamborghini" and x["modell"] == "Urus" and (x["titel"].lower().find("performante") != -1):
        return "Urus Performante"
    else:
        return x["variante"]

def amend_leistung_col_lamborghini_urus_stg_1(x):
    if x["variante"] == "Urus Performante":
        return 666
    else:
        return x["leistung"]

def amend_variante_col_lamborghini_urus_stg_2(x):
    if x["marke"] == "Lamborghini" and x["modell"] == "Urus" and (x["variante"] == "" or x["variante"] is None) and x["leistung"] == 666:
        return "Urus S"
    else:
        return x["variante"]

def amend_variante_col_lamborghini_urus_stg_3(x):
    if x["marke"] == "Lamborghini" and x["modell"] == "Urus" and (x["variante"] == "" or x["variante"] is None) and (x["leistung"] == 650 or x["leistung"] == 649 or x["leistung"] == 662):
        return "Urus"
    else:
        return x["variante"]

def amend_leistung_col_lamborghini_urus_stg_2(x):
    if x["variante"] == "Urus":
        return 650
    else:
        return x["leistung"]

# Apply the functions
df_clean_7["variante"] = df_clean_7.apply(lambda x: amend_variante_col_lamborghini_urus_stg_1(x), axis=1)
df_clean_7["leistung"] = df_clean_7.apply(lambda x: amend_leistung_col_lamborghini_urus_stg_1(x), axis=1)
df_clean_7["variante"] = df_clean_7.apply(lambda x: amend_variante_col_lamborghini_urus_stg_2(x), axis=1)
df_clean_7["variante"] = df_clean_7.apply(lambda x: amend_variante_col_lamborghini_urus_stg_3(x), axis=1)
df_clean_7["leistung"] = df_clean_7.apply(lambda x: amend_leistung_col_lamborghini_urus_stg_2(x), axis=1)
df_clean_7

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,Urus,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Violett Metallic,319900,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,Urus,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Gelb Metallic,299900,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,Urus,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Silber Metallic,299900,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,Urus,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,284990,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,Urus,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Orange,269800,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,Urus Performante,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Silber Metallic,399800,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,Urus S,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Orange Metallic,356999,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,Urus S,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Gelb Metallic,355000,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,Urus,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,398890,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


In [148]:
df_clean_7["leistung"].value_counts()

leistung
650.0    133
666.0     85
780.0      1
746.0      1
809.0      1
789.0      1
782.0      1
Name: count, dtype: int64

#### New column `ausstattung`

In [149]:
# Neue Spalte D = Wenn marke Lamborghini, und model Urus, und entweder im Titel Capsu oder fahrzeugbeschreibung Capsu steht, dann ändere ausstattung auf Pearl Capsule
# Wenn aussattung Pearl Capsule, und entweder variante leere oder leistung leere, dann ändere Variante Urus und Leistung 650

df_clean_8 = df_clean_7.copy()
def add_ausstattung_col_lamborghini_urus_stg_1(x):
    if x["marke"] == "Lamborghini" and x["modell"] == "Urus"\
    and (x["titel"].lower().find("capsu") != -1 or x["fahrzeugbeschreibung_mod"].lower().find("capsu") != -1):
        return "Pearl Capsule"
    else:
        return None

def amend_variante_col_lamborghini_urus_stg_2(x):
    if x["ausstattung"] == "Pearl Capsule" and\
    (x["variante"] == "" or x["variante"] is None or x["leistung"] == "" or x["leistung"] is None):
        return "Urus"
    else:
        return x["variante"]

def amend_leistung_col_lamborghini_urus_stg_3(x):
    if x["ausstattung"] == "Pearl Capsule" and\
    (x["variante"] == "" or x["variante"] is None or x["leistung"] == "" or x["leistung"] is None):
        return 650
    else:
        return x["leistung"]

# Apply the functions
df_clean_8["ausstattung"] = df_clean_8.apply(lambda x: add_ausstattung_col_lamborghini_urus_stg_1(x), axis=1)
df_clean_8["variante"] = df_clean_8.apply(lambda x: amend_variante_col_lamborghini_urus_stg_2(x), axis=1)
df_clean_8["leistung"] = df_clean_8.apply(lambda x: amend_leistung_col_lamborghini_urus_stg_3(x), axis=1)

# Move the Austattung column to be between "variante" and "titel"
austattung_col = df_clean_8.pop("ausstattung")
df_clean_8.insert(3, "ausstattung", austattung_col)

df_clean_8

,marke,modell,variante,ausstattung,titel,form,fahrzeugzustand,leistung,getriebe,farbe,...,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Lamborghini,Urus,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Violett Metallic,...,15900.0,03/2022,2.0,DE,Außenfarbe: Viola Mithras\nInnenausstattung: L...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Viola Mithras\nInnenausstattung: L...
1,Lamborghini,Urus,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Gelb Metallic,...,39400.0,01/2022,2.0,DE,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Giallo Inti (Gelb Metallic)\nInnen...
2,Lamborghini,Urus,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Silber Metallic,...,24000.0,07/2020,2.0,DE,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Grigio Nimbus Matt\nInnenausstattu...
3,Lamborghini,Urus,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,...,25000.0,10/2020,1.0,DE,Außenfarbe: Nero Helene\nInnenausstattung: Alc...,https://suchen.mobile.de/fahrzeuge/details.htm...,5,19,2024-03-16 21:44:09.106746+00:00,Außenfarbe: Nero Helene\nInnenausstattung: Alc...
4,Lamborghini,Urus,Urus,None,Lamborghini Urus,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Orange,...,79800.0,02/2018,2.0,DE,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,19,2024-03-16 21:44:09.106746+00:00,2.Hand.Unfallfrei.Scheckheft gepflegt bei Lamb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Lamborghini,Urus,Urus Performante,None,Lamborghini Urus URUS PERFORMANTE AKRAPOVIC CA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Silber Metallic,...,10.0,None,NaN,DE,Tauchen Sie ein in die Welt grenzenloser Leist...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Tauchen Sie ein in die Welt grenzenloser Leist...
219,Lamborghini,Urus,Urus S,None,"Lamborghini Urus URUS S Pano 23"" Off Road B&O ...",Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Orange Metallic,...,0.0,None,NaN,DE,"EXPORT: 300.000 EURO\nNew Model 2024, Producti...",https://suchen.mobile.de/fahrzeuge/details.htm...,18,19,2024-03-16 21:44:09.106746+00:00,"EXPORT: 300.000 EURO\nNew Model 2024, Producti..."
220,Lamborghini,Urus,Urus S,None,Lamborghini Urus URUS S SPORT ALCANTARA B&O MA...,Super Sport Utility Vehicle,Unfallfrei,666.0,8-Gang-Automatik-Getriebe,Gelb Metallic,...,10.0,None,NaN,DE,Entdecken Sie die unvergleichliche Kraft und E...,https://suchen.mobile.de/fahrzeuge/details.htm...,19,19,2024-03-16 21:44:09.106746+00:00,Entdecken Sie die unvergleichliche Kraft und E...
221,Lamborghini,Urus,Urus,None,Lamborghini Urus URUS TOP CAR DESIGN - EXT. / ...,Super Sport Utility Vehicle,Unfallfrei,650.0,8-Gang-Automatik-Getriebe,Schwarz Metallic,...,12500.0,07/2022,1.0,DE,www.maximum-performance.luxury\ninfo@maximum-p...,https://suchen.mobile.de/fahrzeuge/details.htm...,9,19,2024-03-16 21:44:09.106746+00:00,www.maximum-performance.luxury\ninfo@maximum-p...


In [150]:
df_clean_8["ausstattung"].value_counts()

ausstattung
Pearl Capsule    10
Name: count, dtype: int64

In [151]:
# Save the data frame to a CSV file
df_clean_8.to_csv("lamborghini_urus_2024-03-16.csv", index=False)

### Data Cleaning Aston Martin DBX

In [ ]:
query = """
    SELECT *
    FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`
    WHERE crawled_timestamp = (SELECT MAX(crawled_timestamp) FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`)
"""
df = pd.DataFrame(bq_client.query(query).to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type="tqdm_notebook"))

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
print(f"The length of the dataset is: {len(df)}")

The length of the dataset is: 52479


#### Filter for Aston Martin DBX in DE

In [190]:
df_clean_1 = df[
    (df["marke"]=="Aston Martin") &
    (df["modell"]=="DBX") &
    (df["standort"].str.lower().str.contains("de"))
].reset_index(drop=True)

In [191]:
df_clean_1

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Aston Martin,DBX,None,Aston Martin DBX,SUV / Geländewagen / Pickup,None,551.0,Automatik,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00
1,Aston Martin,DBX,None,Aston Martin DBX,SUV / Geländewagen / Pickup,None,551.0,Automatik,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00
2,Aston Martin,DBX,None,Aston Martin DBX 707,"SUV / Geländewagen / Pickup, Neufahrzeug",Unfallfrei,707.0,Automatik,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00
3,Aston Martin,DBX,None,Aston Martin DBX 707,"SUV / Geländewagen / Pickup, Neufahrzeug",Unfallfrei,707.0,Automatik,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00
4,Aston Martin,DBX,None,Aston Martin DBX 707,"SUV / Geländewagen / Pickup, Vorführfahrzeug",Unfallfrei,707.0,Automatik,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,None,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...","SUV / Geländewagen / Pickup, Vorführfahrzeug",Unfallfrei,707.0,Automatik,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00
73,Aston Martin,DBX,None,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,SUV / Geländewagen / Pickup,None,551.0,Automatik,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00
74,Aston Martin,DBX,None,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,SUV / Geländewagen / Pickup,None,551.0,Automatik,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00
75,Aston Martin,DBX,None,"Aston Martin DBX 707 €316.770,- black/black CA...",SUV / Geländewagen / Pickup,Unfallfrei,707.0,Automatik,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00


#### Add fahrzeugbeschreibung_mod column to replace None values in that column with an empty string

In [192]:
# Add fahrzeugbeschreibung_mod column to replace None values in that column with an empty string
df_clean_1["fahrzeugbeschreibung_mod"] = df_clean_1["fahrzeugbeschreibung"].apply(lambda x: "" if x is None else x)

#### Amend the `form` column

In [193]:
# Spalte E = form = Wenn SUV/Geländewagen/Pickup, oder SUV/Geländewagen/Pickup, Jahreswagen, oder SUV/Geländewagen/Pickup, Neufahrzeug, oder SUV/Geländewagen/Pickup, Tageszulassung, oder SUV/Geländewagen/Pickup, Vorführfahrzeug, oder Andere, oder Andere, Neufahrzeug, oder Sportwagen/Coupe, oder Sportwagen/Coupe, Neufahrzeug, oder Sportwagen/Coupe, Vorführfahrzeug abgebildet wird, dann auf „SUV“ ändern!
df_clean_2 = df_clean_1.copy()

# Write a function that applies the logic above
def amend_form_col_aston_martin_dbx(x):
    if x.lower().find("suv") != -1 or x.lower().find("geländewagen") != -1\
    or x.lower().find("pickup") != -1 or x.lower().find("andere") != -1\
    or x.lower().find("sportwagen") != -1 or x.lower().find("coup") != -1:
        return "SUV"
    else:
        return x

# Apply the function amend_form_col on form using the apply method
df_clean_2["form"] = df_clean_2["form"].apply(amend_form_col_lamborghini_urus)

# Display the data frame
df_clean_2

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,None,551.0,Automatik,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...
1,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,None,551.0,Automatik,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...
2,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
3,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
4,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,None,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...",Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Das Fahrzeug steht aktuell auf Winterrädern (2...
73,Aston Martin,DBX,None,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,Super Sport Utility Vehicle,None,551.0,Automatik,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh..."
74,Aston Martin,DBX,None,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,Super Sport Utility Vehicle,None,551.0,Automatik,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Sehr gerne laden wir Sie zu einer unverbindlic...
75,Aston Martin,DBX,None,"Aston Martin DBX 707 €316.770,- black/black CA...",Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...


In [194]:
df_clean_2["form"].value_counts()

form
Super Sport Utility Vehicle    77
Name: count, dtype: int64

#### Amend the `fahrzeugzustand` column

In [195]:
# Spalte F = fahrzeugzustand = Wenn Unfallfrei, Nicht fahrtauglich, oder (Leere) abgebildet wird, dann auf “Unfallfrei“ ändern
df_clean_3 = df_clean_2.copy()

def amend_fahrzeugzustand_col_aston_martin_dbx(x):
    if x == "" or x is None:
        return "Unfallfrei"
    elif x.lower().find("unfallfrei") != -1:
        return "Unfallfrei"
    else:
        return x
df_clean_3["fahrzeugzustand"] = df_clean_3["fahrzeugzustand"].apply(amend_fahrzeugzustand_col_aston_martin_dbx)
df_clean_3

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...
1,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...
2,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
3,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
4,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,None,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...",Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Das Fahrzeug steht aktuell auf Winterrädern (2...
73,Aston Martin,DBX,None,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh..."
74,Aston Martin,DBX,None,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Sehr gerne laden wir Sie zu einer unverbindlic...
75,Aston Martin,DBX,None,"Aston Martin DBX 707 €316.770,- black/black CA...",Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...


In [196]:
df_clean_3["fahrzeugzustand"].value_counts()

fahrzeugzustand
Unfallfrei                   76
Reparierter Unfallschaden     1
Name: count, dtype: int64

#### Amend the `kilometer` column

In [197]:
# Spalte K = kilometer = Wenn (Leere) abgebildet wird, dann auf „1“ ändern. (Info: Sind oftmals Neuwagen und haben Werkskilometer)
df_clean_4 = df_clean_3.copy()
def amend_kilometer_col_aston_martin_dbx(x):
    if x == "" or x is None:
        return 1
    else:
        return x
df_clean_4["kilometer"] = df_clean_4["kilometer"].apply(amend_kilometer_col_aston_martin_dbx)
df_clean_4

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...
1,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...
2,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
3,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
4,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,None,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...",Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Das Fahrzeug steht aktuell auf Winterrädern (2...
73,Aston Martin,DBX,None,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh..."
74,Aston Martin,DBX,None,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,Super Sport Utility Vehicle,Unfallfrei,551.0,Automatik,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Sehr gerne laden wir Sie zu einer unverbindlic...
75,Aston Martin,DBX,None,"Aston Martin DBX 707 €316.770,- black/black CA...",Super Sport Utility Vehicle,Unfallfrei,707.0,Automatik,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...


#### Amend the `Getriebe` col

In [198]:
# Spalte K = kilometer = Wenn (Leere) abgebildet wird, dann auf „1“ ändern. (Info: Sind oftmals Neuwagen und haben Werkskilometer)
df_clean_5 = df_clean_4.copy()
def amend_getriebe_col_stg_1_aston_martin_dbx(x):
    if x == "" or x is None:
        return "9-Gang-Automatik-Getriebe"
    elif x.lower().find("automatik") != -1:
        return "9-Gang-Automatik-Getriebe"
    else:
        return x
    
df_clean_5["getriebe"] = df_clean_5["getriebe"].apply(amend_getriebe_col_stg_1_aston_martin_dbx)
df_clean_5

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...
1,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...
2,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
3,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
4,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,None,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...",Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Das Fahrzeug steht aktuell auf Winterrädern (2...
73,Aston Martin,DBX,None,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh..."
74,Aston Martin,DBX,None,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Sehr gerne laden wir Sie zu einer unverbindlic...
75,Aston Martin,DBX,None,"Aston Martin DBX 707 €316.770,- black/black CA...",Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...


In [199]:
df_clean_5["getriebe"].value_counts()

getriebe
9-Gang-Automatik-Getriebe    77
Name: count, dtype: int64

#### Amend the `Marke` col based on the `titel` col

In [200]:
# Spalte D -> A = titel = Wenn entweder im titel „gepanzert“ oder fahrzeugbeschreibung „gepanzert“ steht, dann ändere marke (Spalte A) auf “Aston Martin/Trasco“. (Info: Tuning Autos werden nicht berücksichtigt)
# Spalte D -> A = titel = Wenn entweder im titel „Mansory“oder fahrzeugbeschreibung Mansory steht, dann ändere marke (Spalte A) auf “Aston Martin/Mansory“.
df_clean_6 = df_clean_5.copy()
def amend_marke_col_aston_martin_dbx(x, y, replacement_word):
    if x["titel"].lower().find(y.lower()) != -1 or x["fahrzeugbeschreibung_mod"].lower().find(y.lower()) != -1:
        return "Aston Martin | " + replacement_word
    else:
        return x["marke"]

aston_martin_marke_dict = {
    "gepanzert": "Trasco",
    "mansory": "Mansory"
}

for key, value in aston_martin_marke_dict.items():
    df_clean_6["marke"] = df_clean_6.apply(lambda x: amend_marke_col_aston_martin_dbx(x, key, value), axis=1)

df_clean_6

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...
1,Aston Martin,DBX,None,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...
2,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
3,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
4,Aston Martin,DBX,None,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,None,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...",Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Das Fahrzeug steht aktuell auf Winterrädern (2...
73,Aston Martin,DBX,None,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh..."
74,Aston Martin,DBX,None,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,Super Sport Utility Vehicle,Unfallfrei,551.0,9-Gang-Automatik-Getriebe,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Sehr gerne laden wir Sie zu einer unverbindlic...
75,Aston Martin,DBX,None,"Aston Martin DBX 707 €316.770,- black/black CA...",Super Sport Utility Vehicle,Unfallfrei,707.0,9-Gang-Automatik-Getriebe,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...


In [201]:
df_clean_6["marke"].value_counts()

marke
Aston Martin    77
Name: count, dtype: int64

In [202]:
# Spate A|B|D|G|O verändern Spalte C= variante = Wenn marke Aston Martin , und Modell DBX, und entweder im titel 707 oder in fahrzeugbeschreibung 707 steht, dann ändere Variante auf DBX707.
# Spalte G = Leistung = Wenn variante DBX707, und (leere), dann auf 707 ändern

# Spate A|B|D|G|O verändern Spalte C= variante = Wenn marke Aston Martin , und Modell DBX, und nicht variante DBX707, dann ändere Variante auf DBX V8
# Spalte G = Leistung = Wenn variante DBX V8, dann auf 550 ändern (Info: Händler geben immeer 549 oder 551 PS auf Mobile.de an)

df_clean_7 = df_clean_6.copy()
def amend_variante_col_aston_martin_dbx_stg_1(x):
    if x["marke"] == "Aston Martin" and x["modell"] == "DBX" and (x["titel"].lower().find("707") != -1 or x["fahrzeugbeschreibung_mod"].lower().find("707") != -1):
        return "DBX707"
    else:
        return x["variante"]

def amend_leistung_col_aston_martin_dbx_stg_1(x):
    if x["variante"] == "DBX707" or x["variante"] == "" or x["variante"] is None:
        return 707
    else:
        return x["leistung"]

def amend_variante_col_aston_martin_dbx_stg_2(x):
    if x["marke"] == "Aston Martin" and x["modell"] == "DBX" and x["variante"] != "DBX707":
        return "DBX V8"
    else:
        return x["variante"]

def amend_leistung_col_aston_martin_dbx_stg_2(x):
    if x["variante"] == "DBX V8":
        return 550
    else:
        return x["leistung"]

# Apply the functions
df_clean_7["variante"] = df_clean_7.apply(lambda x: amend_variante_col_aston_martin_dbx_stg_1(x), axis=1)
df_clean_7["leistung"] = df_clean_7.apply(lambda x: amend_leistung_col_aston_martin_dbx_stg_1(x), axis=1)
df_clean_7["variante"] = df_clean_7.apply(lambda x: amend_variante_col_aston_martin_dbx_stg_2(x), axis=1)
df_clean_7["leistung"] = df_clean_7.apply(lambda x: amend_leistung_col_aston_martin_dbx_stg_2(x), axis=1)
df_clean_7

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp,fahrzeugbescheibung_mod
0,Aston Martin,DBX,DBX V8,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,550,9-Gang-Automatik-Getriebe,Schwarz Metallic,139900,46095.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03609\nMedia:\nNavigation\nPrem...
1,Aston Martin,DBX,DBX V8,Aston Martin DBX,Super Sport Utility Vehicle,Unfallfrei,550,9-Gang-Automatik-Getriebe,Blau Metallic,144900,25291.0,12/2021,NaN,DE,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr.: V03932\nMedia:\nNavigation\nRadi...
2,Aston Martin,DBX,DBX707,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707,9-Gang-Automatik-Getriebe,Rot Metallic,261888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
3,Aston Martin,DBX,DBX707,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707,9-Gang-Automatik-Getriebe,Weiß,264888,0.0,None,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,6,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
4,Aston Martin,DBX,DBX707,Aston Martin DBX 707,Super Sport Utility Vehicle,Unfallfrei,707,9-Gang-Automatik-Getriebe,Grün Metallic,249907,9498.0,09/2023,NaN,DE,Vielen Dank für Ihr Interesse an den exklusive...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,7,2024-03-16 21:44:09.106746+00:00,Vielen Dank für Ihr Interesse an den exklusive...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Aston Martin,DBX,DBX707,"Aston Martin DBX 707-Carbon-Sitzbelüft-23"" Zol...",Super Sport Utility Vehicle,Unfallfrei,707,9-Gang-Automatik-Getriebe,Schwarz Metallic,229707,18640.0,10/2022,1.0,DE,Das Fahrzeug steht aktuell auf Winterrädern (2...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Das Fahrzeug steht aktuell auf Winterrädern (2...
73,Aston Martin,DBX,DBX V8,Aston Martin DBX V8 1.Ha DE SCHECKHEFT 1913 Sp...,Super Sport Utility Vehicle,Unfallfrei,550,9-Gang-Automatik-Getriebe,Schwarz Metallic,134889,54500.0,10/2020,1.0,DE,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh...",https://suchen.mobile.de/fahrzeuge/details.htm...,1,7,2024-03-16 21:44:09.106746+00:00,"Historie:\n1.Hand, deutsches Fahrzeug, Scheckh..."
74,Aston Martin,DBX,DBX V8,Aston Martin DBX V8*MATT*LUFTFEDERUNG*LED*PANO...,Super Sport Utility Vehicle,Unfallfrei,550,9-Gang-Automatik-Getriebe,Schwarz,158980,13953.0,05/2022,1.0,DE,Sehr gerne laden wir Sie zu einer unverbindlic...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,7,2024-03-16 21:44:09.106746+00:00,Sehr gerne laden wir Sie zu einer unverbindlic...
75,Aston Martin,DBX,DBX707,"Aston Martin DBX 707 €316.770,- black/black CA...",Super Sport Utility Vehicle,Unfallfrei,707,9-Gang-Automatik-Getriebe,None,228880,12989.0,05/2023,1.0,DE,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...,https://suchen.mobile.de/fahrzeuge/details.htm...,4,7,2024-03-16 21:44:09.106746+00:00,Fahrzeug-Nr. für Kundenanfragen: 231700389\n*e...


In [204]:
df_clean_7["variante"].value_counts()

variante
DBX707    49
DBX V8    28
Name: count, dtype: int64

In [205]:
df_clean_7["leistung"].value_counts()

leistung
707    49
550    28
Name: count, dtype: int64

In [203]:
# Save the data frame to a CSV file
df_clean_7.to_csv("aston_martin_dbx_2024-03-16.csv", index=False)